<a href="https://colab.research.google.com/github/harshitlohani04/kaggle-chronicles/blob/master/House_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'house-prices-advanced-regression-techniques:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F5407%2F868283%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240629%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240629T145431Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D45c5a60fd798a657b29051dbdb7b4dd5e92e160ec99773d5be8ac614c1e284a814d62feccd5bdc93bbe493d41db9ff9476ba9bbf0935d02e8af1efce5896bd1f32e32c5733f6e20b9f8f61aa45516d6074727cb4536b9597ab1089ccdc80d038ac725142277addf94736fdbe464539c4bb1162e54885b53001c8da676bcd6f89be5b0b14172c23f9d04e79c80b4b39004b464e7ca0f12adac2ecd8c8736821c00ace7b731b5059894dcdefd59e3a8c2cc3ae5c53b4fa4828ae9d633aa9729a65d8cc73e807a7c12e484519f6ca4119882cc5065876419bfa4003d64d9969bcfed6b58f67be8e7e6a94baae321c736ad8fcac527164740db32458fd83dcad9562'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn
import pandas as pd

train_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
train_data.head()

## Getting to know the data better

***Checking for only those columns that majorly have null vals in them***

In [ ]:
cols_with_null = []
cols = train_data.columns
for i in range(len(cols)):
    if train_data[cols[i]].isnull().sum() / len(train_data[cols[i]]) >=0.5:
        cols_with_null.append((cols[i], train_data[cols[i]].isnull().sum()))


cols_to_drop = [item[0] for item in cols_with_null]
train_data = train_data.drop(columns = cols_to_drop, axis=1)
test_data = test_data.drop(columns = cols_to_drop, axis=1)
train_data

***Dropping the cols that contain more than 95% of a certain val***

#Tried to use this for knowing the dist of the data in a col but the data is huge to be plotted so idea dropped
def Freq_plot(df):
    # select_dtypes(include=["object", "categorical", "int64"]) --> returns a dataset with columns that contain these types of data.
    
    data_cols = df.select_dtypes(include=["int64"]).columns  # Creating the array of cols
    data_col_len = len(data_cols)
    print(data_col_len)
    
    # Approach used in here is using pandas and plt
    for i, col in enumerate(data_cols, 1):
        plt.subplot(7, 5, i) # Creating subplots for all cols
  
        # df[column].value_counts() --> counts the frequency of the vals in that column
        
        df[col].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
        plt.title(f'Distribution of {col}')
        plt.xlabel(col)
        plt.ylabel('Frequency')
        
    plt.show()
        
Freq_plot(train_data)

> Another way of eliminating the cols that contain more than 95% of a certain val in cols with object datatype

In [ ]:
obj_cols = train_data.select_dtypes(include = ["object"]).columns
# For dropping the cols that contain redundant data >95%
for col in obj_cols:
    max_val = train_data[col].value_counts(normalize = True).max()
    if max_val>0.9:
        train_data = train_data.drop(columns = col, axis=1)
        test_data = test_data.drop(columns = col, axis=1)
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.select_dtypes(include=["int64"])

**Handling missing vals in the float columns**

In [ ]:
from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy = "mean")

imputed_cols_train = si.fit_transform(train_data.select_dtypes(include=["float"]))
imputed_cols_test = si.fit_transform(test_data.select_dtypes(include=["float"]))

train_data[train_data.select_dtypes(include=["float"]).columns] = pd.DataFrame(imputed_cols_train, columns=train_data.select_dtypes(include=["float"]).columns)
test_data[test_data.select_dtypes(include=["float"]).columns] = pd.DataFrame(imputed_cols_test, columns=test_data.select_dtypes(include=["float"]).columns)
test_data.select_dtypes(include=["float"])

In [ ]:
obj_cols = train_data.select_dtypes(include=["object"]).columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in obj_cols:
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.fit_transform(test_data[col])
test_data

# Now that all columns have been non-nullified we seperate the features and the prediction col

In [ ]:
Y = train_data["SalePrice"]
X = train_data.drop(columns = ["Id", "SalePrice"], axis=1)

ids = test_data["Id"]
test_data = test_data.drop(columns = ["Id"], axis=1)

**Splitting the data into training and test split**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_Xtrain = scaler.fit_transform(X_train)
scaled_Xtest = scaler.transform(X_test)

scaler_test = StandardScaler()
test_matrix = scaler_test.fit_transform(test_data)

***Creating the model for prediction***

In [ ]:
# Trying out ridge regression and lasso regression
from sklearn.linear_model import Ridge, Lasso

ls = Lasso(alpha = 2.84142) # Producing A BETTER MODEL than ridge regression
# rdg = Ridge(alpha = 5)
ls.fit(scaled_Xtrain, Y_train)
pred = ls.predict(test_matrix)
prediction_arr = []
for i in range(len(ids)):
    prediction_arr.append([ids[i], pred[i]])
final_pred = pd.DataFrame(prediction_arr, columns = ["Id", "SalePrice"])
final_pred.to_csv("submission.csv", index=False)
final_pred